In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from keras import regularizers, layers, optimizers, initializers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.applications import EfficientNetV2M, ConvNeXtBase
import numpy as np
import gc
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(f'Current GPU allocator: {os.getenv("TF_GPU_ALLOCATOR")}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f'Setting memory growth for {gpu}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    

2024-05-05 16:24:41.775155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 16:24:42.812948: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/tobias/miniconda3/env

Current GPU allocator: cuda_malloc_async
Setting memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-05-05 16:24:44.262528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 16:24:44.262596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
study_name = '429_convnextBase_003_998_1'

In [3]:

mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

In [4]:
pd.set_option('display.max_rows', None)

# Aseta näyttämään rajoittamaton määrä sarakkeita
pd.set_option('display.max_columns', None)

In [5]:

pickle_file_path = './data/test_convnextbase_df_003_998.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = './data/train_convnextbase_df_003_998.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
# # FOR TESTING IMAGE AUGEMENTATION
# train_df = train_df.sample(1000)
# test_df = test_df.sample(1000)

In [6]:
feat = pd.read_csv('./data/test.csv')
FEATURE_COLS = feat.columns[1:].tolist()

In [ ]:
# train_images_path = './data/train_images/'
# test_images_path = './data/test_images/'    

# train_df['image_path'] = train_df['id'].apply(lambda x: os.path.join(train_images_path, f'{x}.jpeg'))
# test_df['image_path'] = test_df['id'].apply(lambda x: os.path.join(test_images_path, f'{x}.jpeg'))

In [ ]:
# for column in mean_columns:
#     lower_quantile = train_df[column].quantile(0.005)
#     upper_quantile = train_df[column].quantile(0.985)  
#     train_df = train_df[(train_df[column] >= lower_quantile) & (train_df[column] <= upper_quantile)]

In [ ]:
# from sklearn.model_selection import StratifiedKFold

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for i, trait in enumerate(mean_columns):

#     # Determine the bin edges dynamically based on the distribution of traits
#     bin_edges = np.percentile(train_df[trait], np.linspace(0, 100, 5 + 1))
#     train_df[f"bin_{i}"] = np.digitize(train_df[trait], bin_edges)

# # Concatenate the bins into a final bin
# train_df["final_bin"] = (
#     train_df[[f"bin_{i}" for i in range(len(mean_columns))]]
#     .astype(str)
#     .agg("".join, axis=1)
# )

# # Perform the stratified split using final bin
# train_df = train_df.reset_index(drop=True)
# for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df["final_bin"])):
#     train_df.loc[valid_idx, "fold"] = fold

In [7]:
scaler_feat = RobustScaler()

train_original = train_df.copy()
train_plot = train_df.copy()
sample_df = train_df.copy()

train_df = sample_df[sample_df.fold != 1]
valid_df = sample_df[sample_df.fold == 1]

print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")


# Num Train: 43088 | Num Valid: 10772


In [8]:
import glob

directory_path = './NN_search'
pattern = f"{directory_path}/{study_name}*.h5"

files = glob.glob(pattern)

max_r2_score = float('-inf')
best_model = None

# Käy läpi jokainen tiedosto ja etsi suurin r2_score_inv
for file in files:
    value = float(file.split('best_val')[1].split('_')[1])
    if value > max_r2_score:
        max_r2_score = value
        best_model = file


# Tulosta suurin löydetty r2_score_inv ja vastaava tiedosto
print(f"Best R2-score: {max_r2_score:.5f}")
if best_model:
    print(f"Best model: {best_model}")
else:
    print("No best model found")

best_log_transforms_name =  f'./NN_search/{study_name}_{max_r2_score:.5f}_best_log_transforms.pickle'
best_scalers_name = f'./NN_search/{study_name}_{max_r2_score:.5f}_best_scalers.pickle'

print(f'Opening log transforms from {best_log_transforms_name}')
with open(best_log_transforms_name, 'rb') as f:
    log_transforms = pickle.load(f)

print(f'Opening scalers from {best_scalers_name}')
with open(best_scalers_name, 'rb') as f:
    scaler_transforms = pickle.load(f)


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')
    
custom_objects = {"r2_score_tf": r2_score_tf}

nas_model  = tf.keras.models.load_model(best_model, custom_objects=custom_objects)

nas_model.summary()
        

Best R2-score: 0.39703
Best model: ./NN_search/429_convnextBase_003_998_1_best_val_0.39703_model.h5
Opening log transforms from ./NN_search/429_convnextBase_003_998_1_0.39703_best_log_transforms.pickle
Opening scalers from ./NN_search/429_convnextBase_003_998_1_0.39703_best_scalers.pickle


2024-05-05 16:24:58.936479: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 16:24:58.936584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 16:24:58.936632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 16:24:59.112248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 16:24:59.112345: I tensorflow/compile

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_avg (InputLayer)      [(None, 1024)]            0         
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 2694)              2761350   
                                                                 
 dropout_1 (Dropout)         (None, 2694)              0         
                                                                 
 dense_1 (Dense)             (None, 963)               2595285   
                                                             

In [ ]:
nas_features = nas_model.layers[-2].output
nas_features = Model(inputs=nas_model.input, outputs=nas_features)

In [ ]:
for layer in nas_features.layers:
    layer.trainable = False

In [ ]:
nas_features.summary()

In [ ]:

trainable_count_nas = sum([tf.size(v).numpy() for v in nas_model.trainable_weights])
non_trainable_count_nas = sum([tf.size(v).numpy() for v in nas_model.non_trainable_weights])
print(f"Total parameters nas: {trainable_count_nas + non_trainable_count_nas:,}")
print(f"Trainable parameters nas: {trainable_count_nas:,}")
print(f"Non-trainable parameters nas: {non_trainable_count_nas:,}")

In [ ]:
print(f'Scaler are: {scaler_transforms}')
print(f'Log transforms are: {log_transforms}')

In [9]:
def augment_image(img):

    img = img / 255.0
    
    img = tf.image.random_flip_left_right(img)
    
    img = tf.image.rot90(img, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_hue(img, max_delta=0.1)
    img = tf.image.random_saturation(img, lower=0.9, upper=1.1)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)

    img = tf.image.random_jpeg_quality(img, min_jpeg_quality=85, max_jpeg_quality=100)


    crop_size = tf.random.uniform(shape=[], minval=420, maxval=480, dtype=tf.int32)
    img = tf.image.random_crop(img, size=[crop_size, crop_size, 3])
    img = tf.image.resize(img, [480, 480]) 


    angle = tf.random.uniform([], minval=-np.pi/8, maxval=np.pi/8, dtype=tf.float32)
    img = tfa.image.rotate(img, angles=angle)
    
    img = tf.image.resize(img, [480, 480]) 
    img = img * 255.0

    return img


def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    img = augment_image(img)  
    # img = tf.cast(img, tf.uint8)    

    # tf.print("Final min and max in process_images:", tf.reduce_min(img), tf.reduce_max(img))
    # tf.print("Image type: ", img.dtype)

    return img

def process_image_valid(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    # tf.print("Final min and max in process_image_valid:", tf.reduce_min(img), tf.reduce_max(img))
    # img = tf.cast(img, tf.uint8)
    return img

# Define your dataset processing function
def process_path_train(file_path, targets):
    img = process_image(file_path)
    return img, targets


def process_path_valid(file_path, targets):
    img = process_image_valid(file_path)
    return img, targets

def process_path_test(file_path, dummy):
    img = process_image_valid(file_path)    
    return img, dummy


In [10]:
y_train = train_df[mean_columns]
y_valid = valid_df[mean_columns]


y_train_transformed = y_train.copy()
y_valid_transformed = y_valid.copy()

for target, log_base in log_transforms.items():

    y_train_transformed[target] = np.log(y_train[target]) / np.log(log_base)
    y_valid_transformed[target] = np.log(y_valid[target]) / np.log(log_base)


y_train_transformed = scaler_transforms.fit_transform(y_train_transformed)
y_valid_transformed = scaler_transforms.transform(y_valid_transformed)



In [11]:
BUFFER_SIZE = 55000 
EPOCHS = 12
BATCH_SIZE = 6

train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, y_train_transformed))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, y_valid_transformed))

train_dataset = train_dataset.map(process_path_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

len_dataset = len(train_dataset) 
len_all_train = len(train_dataset) * EPOCHS

print(f'Len dataset: {len_dataset}')
print(f'Len all train: {len_all_train}')



Len dataset: 7182
Len all train: 86184


In [12]:
class CustomCosineDecayWithWarmup(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, alpha=0.0, warmup_learning_rate=0.0, warmup_steps=0, name=None):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.alpha = alpha
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.name = name

    def __call__(self, step):
        with tf.name_scope(self.name or "CustomCosineDecayWithWarmup"):
            # Lämpenemisvaihe
            learning_rate = tf.cond(
                step < self.warmup_steps,
                lambda: self.warmup_learning_rate + step / self.warmup_steps * (self.initial_learning_rate - self.warmup_learning_rate),
                lambda: self.initial_learning_rate
            )
            # Kosinilasku lämpenemisen jälkeen
            cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
                initial_learning_rate=self.initial_learning_rate,
                decay_steps=self.decay_steps,
                alpha=self.alpha
            )
            decayed_learning_rate = cosine_decay(step - self.warmup_steps)
            
            return tf.cond(step < self.warmup_steps, lambda: learning_rate, lambda: decayed_learning_rate)

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_steps": self.decay_steps,
            "alpha": self.alpha,
            "warmup_learning_rate": self.warmup_learning_rate,
            "warmup_steps": self.warmup_steps,
            "name": self.name
        }

In [ ]:
tf.keras.backend.clear_session()
gc.collect()


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')



image_input_avg = Input(shape=(480, 480, 3), name='image_input_avg')

eff_avg_base =  ConvNeXtBase(weights='imagenet', include_top=False, pooling='avg', input_tensor=image_input_avg)

eff_avg_base.trainable = True
for layer in eff_avg_base.layers[:-150]:
    layer.trainable = False

nas_features_layer = nas_features(eff_avg_base.output)
pass_through_layer = Lambda(lambda x: x, name='nas_features')(nas_features_layer)

nas_output = Dense(6, activation='linear', name='final_tune_output')(pass_through_layer)

finetune_model = Model(inputs=image_input_avg, outputs=nas_output, name='finetune_model')

lr_schedule = CustomCosineDecayWithWarmup(
    initial_learning_rate=1e-4,
    decay_steps=len_dataset * (EPOCHS ),
    alpha=0.0,
    warmup_learning_rate=1e-5,
    warmup_steps=len_dataset ,
    name="CosineDecayWithWarmup"
)



# Aseta oppimisnopeuden aikataulu
finetune_model.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule), loss='mae', metrics=['mse', 'mae', 'mape', r2_score_tf])


In [ ]:
trainable_count = sum([tf.size(v).numpy() for v in finetune_model.trainable_weights])
non_trainable_count = sum([tf.size(v).numpy() for v in finetune_model.non_trainable_weights])
print(f"Total parameters: {trainable_count + non_trainable_count:,}")
print(f"Trainable parameters: {trainable_count:,}")
print(f"Non-trainable parameters: {non_trainable_count:,}")

print(f'Total parameters from ConvNext: { (trainable_count + non_trainable_count) - (trainable_count_nas + non_trainable_count_nas):,}')
print(f'Trainable from ConvNext: {trainable_count - trainable_count_nas:,}')
print(f'Non trainable from ConvNext: {non_trainable_count - non_trainable_count_nas:,}')

print(f'Trainable fron NAS: {trainable_count_nas:,}')
print(f'Non trainable from NAS: {non_trainable_count_nas:,}')
print(F'Total parameters from NAS: {trainable_count_nas + non_trainable_count_nas:,}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import io

class TrainImageLoggingCallback(Callback):
    def __init__(self, log_dir, data):
        super(TrainImageLoggingCallback, self).__init__()
        self.log_dir = log_dir
        self.data = data
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        # Fetch a batch of images
        for imgs, tar in self.data.take(1):  # Adjust depending on your dataset structure
            
            # augmented_images = tf.map_fn(augment_image, imgs)
            augmented_images = tf.cast(imgs, tf.uint8)    
        
            # Prepare the image to write to TensorBoard
            with self.writer.as_default():
                tf.summary.image("Augmented Images", augmented_images, step=epoch, max_outputs=20)

            self.writer.flush()


In [ ]:

log_folder = f"./logs/lr_test/trial_{study_name}_503" # MUUTA!

print(f'Logging tensorboard to {log_folder}')
os.makedirs(log_folder, exist_ok=True)

# Aseta logitiedostojen hakemisto
tensorboard_callback = TensorBoard(log_dir=log_folder, histogram_freq=1, update_freq='batch')

In [ ]:
checkpoint_filepath = './NN_search/model_epoch_{epoch:02d}.h5'

model_checkpoint_callback =  tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch',  # Tallenna jokaisen eepokin jälkeen
    verbose=1,  # Näytä viesti tallennuksesta
    save_best_only=False  # Tallenna jokainen eepokki, älä vain parasta
)

callbacks = [
    model_checkpoint_callback,
    tensorboard_callback,
    TrainImageLoggingCallback(log_folder, train_dataset)    
]

history = finetune_model.fit(train_dataset, validation_data=valid_dataset, epochs=EPOCHS, verbose=1, callbacks=callbacks)




In [13]:


class LayerScale(layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tf.Variable(
            self.init_values * tf.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

class StochasticDepth(layers.Layer):
    """Stochastic Depth module.

    It performs batch-wise dropping rather than sample-wise. In libraries like
    `timm`, it's similar to `DropPath` layers that drops residual paths
    sample-wise.

    References:
      - https://github.com/rwightman/pytorch-image-models

    Args:
      drop_path_rate (float): Probability of dropping paths. Should be within
        [0, 1].

    Returns:
      Tensor either with the residual path dropped or kept.
    """

    def __init__(self, drop_path_rate, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_rate = drop_path_rate

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_path_rate
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

    def get_config(self):
        config = super().get_config()
        config.update({"drop_path_rate": self.drop_path_rate})
        return config


def r2_score_tf(y_true, y_pred):

        try: 
            ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
            ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
            r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
            r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
            return tf.reduce_mean(tf.maximum(r2, 0.0))
        except Exception as e:
            # print(f'Error in r2_score_tf: {e}')
            return float('-inf')


    
custom_objects = {"r2_score_tf": r2_score_tf, "LayerScale": LayerScale, "StochasticDepth": StochasticDepth, "CustomCosineDecayWithWarmup": CustomCosineDecayWithWarmup}

for epoch in range(12, EPOCHS+1):
    
    tf.keras.backend.clear_session()
    gc.collect()

    model = tf.keras.models.load_model(f'./NN_search/model_epoch_{epoch:02d}.h5', custom_objects=custom_objects)
    valid_pred = model.predict(valid_dataset, verbose=1)

    valid_pred = scaler_transforms.inverse_transform(valid_pred)


    for i, target in enumerate(mean_columns):
        log_base = log_transforms[target]
        valid_pred[:, i] = np.power(log_base, valid_pred[:, i])


    R2_valid = r2_score(y_valid, valid_pred)
    MSE_valid = mean_squared_error(y_valid, valid_pred)
    MAE_valid = mean_absolute_error(y_valid, valid_pred)
    MAPE_valid = mean_absolute_percentage_error(y_valid, valid_pred)
    print(f'Model at epoch {epoch}:\nR2 : {R2_valid:.5f}, MSE : {MSE_valid:.5f}, MAE : {MAE_valid:.5f}, MAPE : {MAPE_valid:.5f}')



2024-05-05 16:25:36.109085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [10772,6]
	 [[{{node Placeholder/_1}}]]
2024-05-05 16:25:36.109406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [10772,6]
	 [[{{node Placeholder/_1}}]]
2024-05-05 16:25:39.701701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-05-05 16:25:40.447809: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1dffadb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:

1796/1796 [==============================] - 393s 213ms/step
Model at epoch 12:
R2 : 0.37422, MSE : 1739709.43855, MAE : 208.50698, MAPE : 1.43812


In [ ]:
finetune_model = tf.keras.models.load_model('./NN_search/model_epoch_09.h5', custom_objects=custom_objects)

In [ ]:
for target, log_base in log_transforms.items():
   
    train_plot[target] = np.log(train_plot[target]) / np.log(log_base)
   
        
train_plot[mean_columns] = scaler_transforms.transform(train_plot[mean_columns])

In [ ]:
print(log_transforms)
print(scaler_transforms)

In [ ]:
train_original[mean_columns].describe()

In [ ]:
train_plot[mean_columns].describe()

In [ ]:
def plot_data(df, columns_names):
    plt.figure(figsize=(15, 3))

    # Setting up a grid of plots with 2 columns
    n_cols = 6
    n_rows = len(columns_names) // n_cols + (len(columns_names) % n_cols > 0)

    for i, col in enumerate(columns_names):
        plt.subplot(n_rows, n_cols, i+1)
        sns.kdeplot(df[col], bw_adjust=0.5, fill=False, color='blue')
        plt.title(f'Distribution of {col}')
        plt.xlabel('Value')
        plt.ylabel('Density')

    plt.tight_layout()
    plt.show()
    


In [ ]:
plot_data(train_original, mean_columns)

In [ ]:
plot_data(train_plot, mean_columns)

In [ ]:
results_training_name = './data/results_finetune_images.pickle'

if os.path.exists(results_training_name):
    results_training = pd.read_pickle(results_training_name)
else:
    columns = ['Train R2', 'Train MSE', 'Train MAE', 'Train MAPE', 'Valid R2', 'Valid MSE', 'Valid MAE', 'Valid MAPE', 'Train preds Desc', 'Valid preds Desc', 'Test preds Desc' , 'Original data Desc' 'Kaggle R2', 'Scalers', 'Log/Pot transforms']
    results_training = pd.DataFrame(columns = columns)
    results_training.index.name = 'Study name'

study_name_result = f'{study_name}_finetuned'

if study_name_result not in results_training.index:    
    results_training.loc[study_name] = [None]*len(results_training.columns)

In [ ]:
# TEST DATA 

tf.keras.backend.clear_session()
gc.collect()

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()

test_images_path = test_df_copy['image_path'].values

dummy_y = np.zeros((len(test_df_copy), 6))

test_dataset = tf.data.Dataset.from_tensor_slices((test_images_path, dummy_y))
test_dataset = test_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

predictions = finetune_model.predict(test_dataset, verbose=1)

predictions = scaler_transforms.inverse_transform(predictions)


for i, target in enumerate(mean_columns):
    print(f'Logpot transforming target: : {target}, log transform : {log_transforms[target]}')
    log_base = log_transforms[target] 
    predictions[:, i] = np.power(log_base, predictions[:, i])
 

# test_preds_desc =  pd.DataFrame(predictions, columns = mean_columns).describe().to_json()
# results_training.at[study_name_result, 'Test preds Desc'] = test_preds_desc 


In [ ]:
train_df[mean_columns].describe()

In [ ]:
predictions_df = pd.DataFrame(predictions, columns = mean_columns)
predictions_df.describe()

In [ ]:
print(predictions.shape)

In [ ]:

target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

submission_df[target_columns] = predictions


In [ ]:
original_data_desc = train_original[mean_columns].describe().to_json()
results_training.at[study_name_result, 'Original data Desc'] = original_data_desc



In [ ]:
print(f'{str(log_transforms.items())}')
print(f'{str(scaler_transforms)}')

results_training.at[study_name_result, 'Scalers'] = f'{scaler_transforms}'
results_training.at[study_name_result, 'Log/Pot transforms'] = f'{str(log_transforms.items())}'



In [ ]:
# results_training.at[study_name_result, 'Kaggle R2'] = None


# # results_training.drop('423_std_powerlog_3', inplace=True)
# # results_training.head()

# for index, row in results_training.iterrows():
#     print(f"Study Name: {index}")
#     print(f'Kaggle R2: {row["Kaggle R2"]}')
#     print(f"Train R2: {row['Train R2']}, Train MSE: {row['Train MSE']}, Train MAE : {row['Train MAE']}, Train MAPE: {row['Train MAPE']}")
#     print(f'Valid R2: {row["Valid R2"]}, Valid MSE: {row["Valid MSE"]}, Valid MAE: {row["Valid MAE"]}, Valid MAPE: {row["Valid MAPE"]}')
#     print("-" * 50)
#     print("Train preds Description:")
#     display(pd.read_json(row['Train preds Desc']))
#     print("Valid preds Description:")
#     display(pd.read_json(row['Valid preds Desc']))
#     print("Test preds Description:")
#     display(pd.read_json(row['Test preds Desc']))
#     print("Original data Description:")
#     display(pd.read_json(row['Original data Desc']))
    

In [ ]:
submission_df.head()

In [ ]:
print(submission_df.info())

submission_df.to_csv('./data/submission.csv', index=False)

In [ ]:
with open(results_training_name, 'wb') as f:
    results_training.to_pickle(f)

In [ ]:
feature_model_output = finetune_model.get_layer('nas_features')
feature_model = Model(inputs=finetune_model.input, outputs=feature_model_output.output)


In [ ]:
train_all_paths = train_original['image_path'].values
dummy_y = np.zeros((len(train_original), 6))

train_all_dataset = tf.data.Dataset.from_tensor_slices((train_all_paths, dummy_y))
train_all_dataset = train_all_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_all_dataset = train_all_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_features = feature_model.predict(train_all_dataset, verbose=1)

train_original[f'model_features_{study_name}'] = train_features.tolist()

In [ ]:
train_original.head()

In [ ]:
pickle_file_path = f'./data/train_{study_name}.pickle'

print(f'Saving train_df to {pickle_file_path}')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(train_original, f)

In [ ]:
test_all_paths = test_df['image_path'].values
dummy_y = np.zeros((len(test_df), 6))

test_all_dataset = tf.data.Dataset.from_tensor_slices((test_all_paths, dummy_y))
test_all_dataset = test_all_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_all_dataset = test_all_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_features = feature_model.predict(test_all_dataset, verbose=1)

test_df[f'model_features_{study_name}'] = test_features.tolist()

In [ ]:
test_df[f'model_features_{study_name}'].head()

In [ ]:
test_df.head()

In [ ]:
pickle_file_path = f'./data/test_{study_name}.pickle'

print(f'Saving test_df to {pickle_file_path}')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(test_df, f)
